In [1]:
# import all of the necessary libraries
import easyocr
import cv2

In [ ]:
def size_floorplan(floorplan):
    """Obtain the size data from the floorplan"""
    reader = easyocr.Reader(['en']) # GPU=false taken out for now
    result = reader.readtext(floorplan)
    
    scraped_text = ''
    for i in result:
        scraped_text += i[1]
        scraped_text += ","
    text_list = scraped_text.split(",")
    
    m_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square meters|square metres|square meter|square metre|square m|square mt|sqmt|sq mt|sq.mt|sq. mt|sqm|sq m|sq.m|sq. m|meters2|metres2|meter2|metre2|mt2|m2)", re.IGNORECASE)
    f_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square feet|square ft|square f|sqft|sq ft|sq.ft|sq. ft|sqf|sq f|sq.f|sq. f|feet2|ft2|f2)", re.IGNORECASE)
    house_size = []
    for x in text_list:
        for match in re.finditer(m_patterns, x):
            #print('%s : %s' % (match.groups()))
            house_size.append(list(match.groups()))
        for match in re.finditer(f_patterns, x):
            #print('%s : %s' % (match.groups())) 
            house_size.append(list(match.groups()))
    for a in house_size:
        try:
            a[0] = int(a[0])
        except ValueError:
            a[0] = float(a[0])
            
    # then here you could find the
    house_size = dict((house_size))
    return house_size


In [ ]:
image = "https://media.rightmove.co.uk/72k/71134/111198221/71134_30837396_FLP_00_0002.jpeg"
reader = easyocr.Reader(['en'], gpu=False)
result = reader.readtext(image)

scraped_text = ''
for i in result:
    scraped_text += i[1]
    scraped_text += ","
    
text = scraped_text.split(",")

r = size_floorplan('https://media.rightmove.co.uk/77k/76792/48779142/76792_HAM547-t-3762_FLP_01_0000.jpg')

# Code to take the largest returned value from 
amount = 0
for size in r.keys():
    if size > amount:
        amount = size
largest_size = {}
largest_size[amount] = r[amount]
largest_size[amount] = largest_size[amount].lower()
print(largest_size)

metre_values = ["square meters", "square metres", "square meter", "square metre", "square m", "square mt", "sqmt", "sq mt",
                "sq.mt", "sq. mt", "sqm", "sq m", "sq.m", "sq. m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
feet_values = ["square feet","square ft","square f","sqft","sq ft","sq.ft","sq. ft","sqf","sq f","sq.f","sq. f","feet2",
               "ft2","f2"]

if largest_size[amount] in metre_values:
    largest_size[amount] = "sq m"
elif largest_size[amount] in feet_values:
    largest_size[amount] = "sq ft"
print(largest_size)